# Flower Recognition with EfficientNetB1
https://www.kaggle.com/alxmamaev/flowers-recognition/home


![](https://1.bp.blogspot.com/-Cdtb97FtgdA/XO3BHsB7oEI/AAAAAAAAEKE/bmtkonwgs8cmWyI5esVo8wJPnhPLQ5bGQCLcBGAs/s1600/image4.png)

![](https://1.bp.blogspot.com/-DjZT_TLYZok/XO3BYqpxCJI/AAAAAAAAEKM/BvV53klXaTUuQHCkOXZZGywRMdU9v9T_wCLcBGAs/s1600/image2.png)

In [ ]:
# GPU usage information

!nvidia-smi

# Setting Random Seed

In [ ]:
from numpy.random import seed
seed(1)
import tensorflow as tf
tf.random.set_seed(2)

# Load Data

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1vYdIDWC5FSaFeslc8HWxWlD7MtBsAz28' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1vYdIDWC5FSaFeslc8HWxWlD7MtBsAz28" -O flowers_dataset.zip && rm -rf /tmp/cookies.txt

In [ ]:
!unzip /content/flowers_dataset.zip &> /dev/null

#Import Necessary Library

In [ ]:
import os
import random
import itertools
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as pyplot
import numpy as np
import pandas as pd

from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
tf.__version__

# Data Preprocessing

In [ ]:
train_dir = "/content/flowers_dataset/train"
val_dir = "/content/flowers_dataset/validate"
test_dir = "/content/flowers_dataset/test"

In [ ]:
image_size = 240
num_class = 5

batch_size = 24

#image augmentation
train_datagen = ImageDataGenerator(
        rescale=1.0/255.0, 
        shear_range = 0.2,
        zoom_range = 0.2,
        fill_mode="nearest",
        vertical_flip=True,
        horizontal_flip=True)
 
validation_datagen = ImageDataGenerator(rescale=1.0/255.0)

#create train and validation 
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(image_size, image_size),
        batch_size=batch_size,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        val_dir,
        target_size=(image_size, image_size),
        batch_size=batch_size,
        class_mode='categorical')

# Data Visualization

In [ ]:
#train statistics
y_train = train_generator.labels
df_train = pd.DataFrame(y_train, columns=['class'])
ax = df_train['class'].value_counts().sort_index(na_position='first').plot(kind='bar', rot=0, figsize=(10,5), sort_columns=True)
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x() + p.get_width()/2, height+4, '{:1.0f}'.format(height), ha="center")

In [ ]:
#validation statistics
y_val = validation_generator.labels
df_val = pd.DataFrame(y_val, columns=['class'])
ax = df_val['class'].value_counts().sort_index(na_position='first').plot(kind='bar', rot=0, figsize=(10,5), sort_columns=True)
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x() + p.get_width()/2, height+1.5, '{:1.0f}'.format(height), ha="center")

In [ ]:
#visualize image from train set

x, y = train_generator.next() # x=image, y=label
i = random.randint(0, batch_size-1)
pyplot.subplots(1, 1, figsize=(10,5))
pyplot.imshow(x[i])

In [ ]:
#visualize image from validation set

x, y = validation_generator.next()
i = random.randint(0, batch_size-1)
pyplot.subplots(1, 1, figsize=(10,5))
pyplot.imshow(x[i])

# Create Model

In [ ]:
# list of models
# https://www.tensorflow.org/api_docs/python/tf/keras/applications/

from tensorflow.keras.applications import EfficientNetB1

efnb1 = EfficientNetB1(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

for layer in efnb1.layers:
  layer.trainable = True

model = Sequential()
model.add(efnb1)
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax'))

model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Train Model

In [ ]:
# callbacks = [
#                ModelCheckpoint('efn1_best_model.h5', monitor='val_accuracy', mode='max', save_best_only=True, verbose=1),
#                ReduceLROnPlateau(monitor='val_accuracy', mode='max', patience=3, verbose=1),
#                EarlyStopping(monitor='val_accuracy', patience=10, mode='max', verbose=1)
#                ]

# history = model.fit(
#     train_generator,
#     steps_per_epoch=len(train_generator),
#     epochs=100,
#     callbacks = callbacks,
#     validation_data = validation_generator,
#     validation_steps=len(validation_generator)
# )

#Plot Train and Validation Graphs

In [ ]:
# #plot loss
# pyplot.figure(figsize=(30,5))
# pyplot.subplot(121)
# pyplot.plot(history.history['loss'])
# pyplot.plot(history.history['val_loss'])
# pyplot.title('Cross Entropy Loss')
# pyplot.ylabel('loss')
# pyplot.xlabel('epoch')
# pyplot.legend(['Train', 'Validate'], loc='upper left')

# #plot accuracy
# pyplot.subplot(122)
# pyplot.plot(history.history['accuracy'])
# pyplot.plot(history.history['val_accuracy'])
# pyplot.title('Classification Accuracy')
# pyplot.ylabel('accuracy')
# pyplot.xlabel('epoch')
# pyplot.legend(['Train', 'Validate'], loc='upper left')
# pyplot.show()

# Test Set Evaluation

In [ ]:
# download pre-trained weight
!wget https://storage.googleapis.com/kao-weights/efn1_best_model.h5

In [ ]:
#implement model
model.load_weights('efn1_best_model.h5')

In [ ]:
#create test set
test_generator = validation_datagen.flow_from_directory(
        test_dir,
        target_size=(image_size, image_size),
        batch_size=1,
        class_mode='categorical')

In [ ]:
#evaluate test set
score = model.evaluate(test_generator, verbose=1)

print('Test loss: %.3f' % (score[0] * 100.0))
print('Test accuracy: %.3f' % (score[1] * 100.0))

In [ ]:
#split imagedatagenerator into x_test and y_test
from platform import python_version_tuple

if python_version_tuple()[0] == '3':
    xrange = range
    izip = zip
    imap = map
else:
    from itertools import izip, imap

x, y = izip(*(test_generator[i] for i in xrange(len(test_generator))))
x_test, y_test = np.vstack(x), np.vstack(y)

In [ ]:
#predict values from test set
y_pred = model.predict(test_generator, steps = len(test_generator))
#convert predictions classes to one hot vectors
y_pred_classes = np.argmax(y_pred, axis=1)
#convert test observations to one hot vectors
y_true = np.argmax(y_test, axis=1)

In [ ]:
#visualize image from test set

i = random.randint(0, len(x_test))
pyplot.subplots(1, 1, figsize=(10,5))
pyplot.imshow(x_test[i])
pyplot.show()

#print target on test image.
idx_x = y_pred_classes[i]
idx_y = list(y_test[i]).index(1)
y_name = test_generator.class_indices
print('prediction is',list(y_name.keys())[list(y_name.values()).index(idx_x)]) 
print('label is',list(y_name.keys())[list(y_name.values()).index(idx_y)]) 

In [ ]:
#plot confusion matrix
def plot_confusion_matrix(cm, classes, cmap=pyplot.cm.Blues):
  pyplot.imshow(cm, interpolation='nearest', cmap=cmap)
  pyplot.title('Confusion Matrix')
  pyplot.colorbar()
  tick_marks = np.arange(len(classes))
  pyplot.xticks(tick_marks, classes)
  pyplot.yticks(tick_marks, classes)

  thresh = cm.mean()
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    pyplot.text(j, i, cm[i, j],
                horizontalalignment="center",
                color="white" if cm[i, j] > thresh else "black")
    pyplot.tight_layout()
    pyplot.ylabel('True Label')
    pyplot.xlabel('Predicted Label')

In [ ]:
#compute confusion matrix 
confusion_mtx = confusion_matrix(y_true, y_pred_classes)
#plot confusion matrix
plot_confusion_matrix(confusion_mtx, classes=range(5))

In [ ]:
#classification report
print(classification_report(y_true, y_pred_classes, target_names=test_generator.class_indices, digits=4))